In [30]:
## basic setup

# install spacy
#!pip install -U spacy
#!python -m spacy download de_core_news_sm

#!pip list

from bs4 import BeautifulSoup

def zeilenumbrüche_entfernen(text):
    try:
        text = text.replace("-\n", "")
        text = text.replace("\n", " ")
    except:
        pass
    return text


with open('files/buechner_danton_1835.TEI-P5.xml', encoding="utf-8") as f:
    read_data = f.read()
    soup = BeautifulSoup(read_data, 'html.parser')

In [31]:
## Worfeldanalyse

import spacy
from collections import Counter

# Load German tokenizer, tagger, parser and NER
nlp = spacy.load("de_core_news_sm")

# stop words einrichten
stop_words = ["ist's", 'o', 'St.']


# langes "s" (ſ) tilgen
def buchstaben_normalisieren(text):
    text = text.replace("ſ", "s")
    text = text.replace("’", "'")
    return text

    
def get_word_count(text):

    # Buchstaben normalisieren
    text = buchstaben_normalisieren(text)
    text = nlp(text)
    # Text filtern -- nur Substantive
    words = [token.lemma_.lower() for token in text if not token.is_stop and not token.is_punct and not token.is_space and (token.pos_ == "NOUN" or token.pos_ == "PROPN")]
    # Text filtern -- alles
    #words = [token.lemma_.lower() for token in text if not token.is_stop and not token.is_punct and not token.is_space]
    
    # stop words filtern
    temp_words = []
    for word in words:
        if word not in stop_words:
            temp_words.append(word)
    words = temp_words
    
    # Häufigkeit für jedes Wort berechnen
    word_freq = Counter(words)
    
    # n häufigste Wörter auswählen und ausgeben
    most_freq_words = word_freq.most_common(60)

    for word in most_freq_words:
        print(word)
    print("-----"*5)

In [32]:
## Ausgabe der Personenliste laut Text
# print(soup.castlist.text)

In [33]:
## Ausgabe aller Dialoge von Person x

dialoge_person = soup.find_all('sp', {'who': '#SIM'})

for i in range(len(dialoge_person)):
    print(dialoge_person[i].text)
    print("____"*5, i+1, "____"*5)

Simon(ſchlägt das Weib).
Du Kuppelpelz, du runzliche Sublimatpille, du
wurmſtichiger Sündenapfel!
____________________ 1 ____________________
Simon.
Nein, laßt mich, Römer! Zerſchellen will ich dies
Geripp! Du Veſtalin!
____________________ 2 ____________________
Simon.
So reiß ich von den Schultern dein Gewand,

Nackt in die Sonne ſchleudr’ ich dann dein Aas,
In jeder Runzel deines Leibes niſtet Unzucht.
(Sie werden getrennt.)
____________________ 3 ____________________
Simon.
Wo iſt die Jungfrau? Sprich! Nein, ſo kann
ich nicht ſagen. Das Mädchen; nein, auch das nicht;
die Frau, das Weib! Auch das, auch das nicht!
Nur noch ein Name; o, der erſtickt mich! Ich
habe keinen Athem dafür.
____________________ 4 ____________________
Simon.
Alter Virginius, verhülle dein kahles Haupt, —
der Rabe Schande ſitzt darauf, und hackt nach dei-
nen Augen. Gebt mir ein Meſſer, Römer!
(Er ſinkt um.)
____________________ 5 ____________________
Simon.
Du biſt die Vampyrzunge, die mein wärmſtes
Herzblut 

In [34]:
## Alle Personen mit Gesamtzahl von Dialogen und Wörtern finden (liste + nested_dict)

personen = soup.find_all('sp')
liste_personen = []

# Einzelne Dialoge der Figuren durchgehen
for auftritt in personen:
    
    # "who"-Tag ermitteln
    who = auftritt.get("who")
    
    # Name ermitteln, Punktzeichen entfernen
    if auftritt.speaker.text[-1] == ".":
        name = auftritt.speaker.text[:-1]
    else:
        name = auftritt.speaker.text
        
    # Redebeitrag suchen (und Zeilenumbrüche entfernen)
    redebeitrag = zeilenumbrüche_entfernen(auftritt.p.text)
        
    # Wenn Person bereits in Liste, alternative Namen ergänzen, Zähltwert erhöhen, Redebeitrag ergänzen
    Wiederholung = False
    for eintrag in liste_personen:
        if who == eintrag["who"]: 
                        
            # Zählwert erhöhen
            eintrag["count"] += 1
            
            # Wiederholung auf True setzen
            Wiederholung = True
            
            # Nach alternativen Namen suchen
            doublette = False
            for variante in eintrag["name"]:
                if name == variante:
                    doublette = True
            if not doublette:
                eintrag["name"].append(name)
                
            # Redebeitrag aufnehmen
            eintrag["rede"].append(redebeitrag)
            eintrag["rede_full"] += (" " + str(redebeitrag))
    
    # Wenn Person noch nicht in Liste, "who" und Name sowie redebeitrag aufnehmen
    if not Wiederholung:
        liste_personen.append({"who": who, "name" : [name], "count" : 1, "rede" : [redebeitrag], "rede_full" : str(redebeitrag)})

# Liste nach Zahl an Dialogen (="count") sortieren
liste_personen = sorted(liste_personen, key=lambda d: d['count'], reverse=True)  

# Ausgabe der Beitragslänge
liste_personen_neu = []
for person in liste_personen:
    if (len(person['rede_full'].split())/person['count']) > 0:
        # Formatierung
        s1 = 42
        for _ in person['name']:
            s1 -= len(_) + 4 
        s2 = 8 - len(person['who'])
        s3 = 3 - len(str(person['count']))
        o4 = len(person['rede_full'].split())
        o4 = " "*(5 - len(str(o4))) + str(o4)
        o5 = len(person['rede_full'].split())/person['count']
        o5 = '{:.1f}'.format(o5)
        o5 = " "*(6 - len(str(o5))) + str(o5)
        
        # Ausgabe
        print(person['name'], " "*s1, person['who'], " "*s2,  " "*s3, person['count'], "|", o4, "|", o5)
        # Ausgabe der Wortfelder
        # get_word_count(person['rede_full'])
# print("Gesamt:", count)
    

['Danton']                                  #DAN       117 |  4439 |   37.9
['Camille']                                 #CAM        46 |  1272 |   27.7
['Lacroix']                                 #LAC        43 |  1059 |   24.6
['Robespierre']                             #ROB        29 |  2289 |   78.9
['Hérault']                                 #HERA       23 |   615 |   26.7
['Simon']                                   #SIM        23 |   309 |   13.4
['St. Juſt']                                #STJU       23 |  1006 |   43.7
['Julie']                                   #JUL        20 |   299 |   14.9
['Barrère']                                 #BAR        17 |   466 |   27.4
['Weib', 'Ein Weib', 'Einige Weiber']       #WEIB       16 |   257 |   16.1
['Erſter Bürger']                           #ERBUER     16 |   244 |   15.2
['Präſident', 'Der Präſident', 'Hermann']   #PRAES      16 |   243 |   15.2
['Ein Bürger', 'Bürger']                    #BUER       14 |   137 |    9.8
['Zweiter Bü

In [35]:
## Statistik der Auftritte

statistik_auftritte = []

# Funktion zur Ausgabe aller Sprecher (Personen in Akt/Auftritt "speaker")
def get_sprecherliste(abschnitt):
    stat_auftritt_sprecher = []
    auftritt_sprecher_liste = {}
    speakers = abschnitt.find_all('sp')
    
    # Sprecherliste erstellen und Redebeiträge zählen
    for speaker in speakers:
        
        who = speaker.get("who")
       
        # Redebeitrag suchen (und Zeilenumbrüche tilgen)
        redebeitrag = zeilenumbrüche_entfernen(speaker.p.text)   
    
        # Sprecher ("who") und Redebeiträge sammeln
        if who in auftritt_sprecher_liste:            
            auftritt_sprecher_liste[who] = [auftritt_sprecher_liste[who][0] + 1, auftritt_sprecher_liste[who][1] + (" " + str(redebeitrag))]
        else:
            auftritt_sprecher_liste[who] = [1, str(redebeitrag)]
    
    # Mit Sprecherliste aus anderer Zelle abgleichen, um Gesamtbeiträge zu erhalten
    for spr,red in auftritt_sprecher_liste.items():
        stat_sprecher = []
        for sprecher in liste_personen:
            if sprecher["who"] == spr:
                nam = sprecher["name"]
                cou = sprecher["count"]
                bei = sprecher["rede_full"]
        
        # Daten an Liste übergeben
        stat_sprecher.append(spr)
        stat_sprecher.append(nam)
        stat_sprecher.append(red[0])
        stat_sprecher.append(cou)
        stat_sprecher.append(red[1])
        stat_sprecher.append(bei)
    
        # Einträge zusammenführen
        stat_auftritt_sprecher.append(stat_sprecher)
    
    # Wordcount aufrufen
    #get_word_count(redebeiträge_abschnitt)
    
    return stat_auftritt_sprecher

    
# Die unterschiedlichen Abschnitte (= Akt (n=1)oder Auftritt (n=2)) suchen
abschnitte = soup.find_all('div', {'n': ['1', '2']})
akt = 0
    
for abschnitt in abschnitte:    
    stat_abschnitt = []
    
    # Untersuchen ob der Abschnitt ein Akt oder eine Szene ist
    div = abschnitt.parent.div.get("n")
    if div == "1":
        akt += 1
        abschnitt_nummer = 0
    elif div == "2":
        abschnitt_nummer += 1
        # Ausgabe der Aktnummer
        stat_abschnitt.append(akt)
        # Ausgabe des Abschnittnamens (="head")
        stat_abschnitt.append(abschnitt.head.text)
        stat_abschnitt.append(abschnitt_nummer)
        # Ausgabe der (ersten) Regieanweisungen (="stage")
        stat_abschnitt.append(zeilenumbrüche_entfernen(abschnitt.stage.text))
    
        # Funktion für Sprecher und Gesprächsbeiträge aufrufen
        stat_abschnitt.append(get_sprecherliste(abschnitt))
        
        # Ergebnisse der Liste "statistik_auftritte" hinzufügen
        statistik_auftritte.append(stat_abschnitt)
        
# Ausgabe der Liste (formatiert)
for elem in statistik_auftritte:
    print("---- Akt", elem[0], ">>", elem[1], "-%d-" %elem[2])
    print("--", elem[3])
    wordcount = ""
    
    # Ausgabe der Figuren mit Redestatistik
    for x in elem[4]:
        # Spatien für Ausgabe
        s1 = 42
        for _ in x[1]:
            s1 -= len(_) + 4
        s2 = 2 - len(str(x[2]))
        s3 = 3 - len(str(x[3]))
        s4 = 3 - len(str(len(x[4].split())))
        s5 = 4 - len(str(len(x[5].split())))
        
        # Ausgabe
        print(x[1], " "*s1, "Dialoge:", " "*s2, x[2], "/", " "*s3, x[3], "| Wörter in Abschnitt:", " "*s4, len(x[4].split()), "/", " "*s5, len(x[5].split()))
        
        # für Wordcount
        wordcount += x[4]
        
    # Ausgabe der n häufigsten Wörter
    #get_word_count(wordcount)
    
    print("_____"*21)


---- Akt 1 >> Erſter Act. -1-
-- Hérault Séchelles, einige Damen (am Spieltiſch), Danton, Julie, ſeine Gattin, (etwas weiter weg, Danton auf einem Schemel zu den Füßen Juliens).
['Danton']                                  Dialoge:  10 /  117 | Wörter in Abschnitt:  355 /  4439
['Julie']                                   Dialoge:   4 /   20 | Wörter in Abschnitt:   11 /   299
['Eine Dame', 'Dame']                       Dialoge:   4 /    4 | Wörter in Abschnitt:   33 /    33
['Hérault']                                 Dialoge:   8 /   23 | Wörter in Abschnitt:  303 /   615
['Camille']                                 Dialoge:   5 /   46 | Wörter in Abschnitt:  241 /  1272
['Philippeau']                              Dialoge:   3 /   12 | Wörter in Abschnitt:  102 /   264
_________________________________________________________________________________________________________
---- Akt 1 >> Eine Gaſſe. -2-
-- Simon. Sein Weib.
['Simon']                                   Dialoge:  12 /   23 |

['Lacroix']                                 Dialoge:   3 /   43 | Wörter in Abschnitt:   36 /  1059
['Hérault']                                 Dialoge:   3 /   23 | Wörter in Abschnitt:   59 /   615
['Danton']                                  Dialoge:  12 /  117 | Wörter in Abschnitt:  432 /  4439
['Camille']                                 Dialoge:  11 /   46 | Wörter in Abschnitt:  266 /  1272
_________________________________________________________________________________________________________
---- Akt 3 >> Platz vor der Conciergerie. -14-
-- Ein Schließer, zwei Fuhrleute mit Karren, Weiber.
['Schließer']                               Dialoge:   3 /    6 | Wörter in Abschnitt:   23 /    50
['Erſter Fuhrmann']                         Dialoge:   5 /    5 | Wörter in Abschnitt:   46 /    46
['Zweiter Fuhrmann']                        Dialoge:   6 /    6 | Wörter in Abschnitt:   98 /    98
['Weib', 'Ein Weib', 'Einige Weiber']       Dialoge:   1 /   16 | Wörter in Abschnitt:    5 / 

In [36]:
## Experimentierzelle - Innen / Außen / Anklage


außen = ["Eine Gaſſe.", "Eine Promenade.", "Strasse vor Danton’s Hauſe.", "Platz vor dem Juſtiz-Palaſte.", "Platz vor der Conciergerie.", "Der Revolutions-Platz."]
innen = ["Ein Zimmer.", "Die Conciergerie.", "Das Luxemburg.", "Der Jacobinerklubb.", "Erſter Act.", "Freies Feld."]
anklage = ["Der National-Convent.", "Das Revolutions-Tribunal.", "Der Wohlfahrts-Ausſchuſs."]

redebeiträge_innen = []
redebeiträge_außen = []
redebeiträge_anklage = []
personen_innen = {}
personen_außen = {}
personen_anklage = {}

# Untersuche alle Abschnitte
for elem in statistik_auftritte:
    for x in elem[4]:
        # Wenn der Abschnitt (elem[1]) außen ist, die sprechende Person (elem[4]) untersuchen
        if elem[1] in außen:
            # Redebeiträge sammeln
            redebeiträge_außen.append(x[5])
            # Personenliste sammeln
            if x[0] in personen_außen:
                personen_außen[x[0]] = (personen_außen[x[0]] + x[2])
            else:
                personen_außen[x[0]] = x[2]
        # Ansonsten entsprechendes für innen durchführen
        elif elem[1] in innen:
            redebeiträge_innen.append(x[5])
            if x[0] in personen_innen:
                personen_innen[x[0]] = (personen_innen[x[0]] + x[2])
            else:
                personen_innen[x[0]] = x[2]
        # Entsprechendes für anklage durchführen
        if elem[1] in anklage:
            # Redebeiträge sammeln
            redebeiträge_anklage.append(x[5])
            # Personenliste sammeln
            if x[0] in personen_anklage:
                personen_anklage[x[0]] = (personen_anklage[x[0]] + x[2])
            else:
                personen_anklage[x[0]] = x[2]

# Wer kommt wie oft wo vor?

# Wer spricht während der Anklage
print("Sprecher in", anklage)
print("---- Anklage", len(" ".join(redebeiträge_anklage).split()))
for person in liste_personen:
    if (person['who']) in personen_anklage:
        print(person['name'], "|", personen_anklage[person['who']]) 

#print('_____________________')
# Wordcount
#get_word_count(" ".join(redebeiträge_anklage))
print("###"*30)


print("---- Außen", len(" ".join(redebeiträge_außen).split()))
# Wenn die Person nur in "außen" vorkommt
for person in liste_personen:    
    if (person['who']) in personen_außen and person['who'] not in personen_innen:
        print(person['name'], "|", personen_außen[person['who']])
        
# Wordcount
#get_word_count(" ".join(redebeiträge_außen))

print("---- Innen", len(" ".join(redebeiträge_innen).split()))
# Wenn die Person nur in "innen" vorkommt
for person in liste_personen:    
    if (person['who']) in personen_innen and person['who'] not in personen_außen:
        print(person['name'], "|", personen_innen[person['who']])
        
# Wordcount       
#get_word_count(" ".join(redebeiträge_innen))


# Sprecherliste ausgeben und mit personen_außen abgleichen (= detailliert)
print("###"*30)
for person in liste_personen:
    print(person['name'])
    if (person['who']) in personen_außen:
        print("außen:", personen_außen[person['who']])
    if (person['who']) in personen_innen:
        print("innen:", personen_innen[person['who']])
        
        




Sprecher in ['Der National-Convent.', 'Das Revolutions-Tribunal.', 'Der Wohlfahrts-Ausſchuſs.']
---- Anklage 16351
['Danton'] | 9
['Camille'] | 1
['Robespierre'] | 1
['St. Juſt'] | 10
['Barrère'] | 17
['Präſident', 'Der Präſident', 'Hermann'] | 7
['Legendre'] | 3
['Fouquier'] | 1
['Ein Anderer', 'Eine Andere'] | 7
['Einige Stimmen'] | 1
['Callot'] | 6
['Schließer'] | 2
['Billaud'] | 5
['Viele der Stimmen', 'Viele Stimmen'] | 1
['Ein Deputirter'] | 2
##########################################################################################
---- Außen 23045
['Simon'] | 23
['Weib', 'Ein Weib', 'Einige Weiber'] | 16
['Erſter Bürger'] | 16
['Ein Bürger', 'Bürger'] | 14
['Zweiter Bürger'] | 13
['Zweiter Herr'] | 7
['Ein Bettler', 'Bettler'] | 6
['Zweiter Fuhrmann'] | 6
['Junger Herr'] | 5
['Dumas'] | 5
['Erſter Fuhrmann'] | 5
['Erſter Herr'] | 4
['Eugenie'] | 4
['Dritter Bürger'] | 3
['Junger Menſch'] | 3
['Soldat'] | 3
['Alle'] | 2
['Bänkelſänger'] | 2
['Madame'] | 2
['Erſter Henker'] | 2
[

In [42]:
## Personenliste in Klassen einteilen

liste_personen_sorted = []

# Klasse "gruppe_volk" definieren
# gruppe_volk = ["#WEIB", "#WEIBKI", "#ERWEIB", "#ZWEWEIB", "#DRIWEIB", "#SIM", "#STIM", "#BET"]
# Klasse "gruppe_deputierte" (= Abgeordnete des Nationalkonvents)
gruppe_deputierte = ["#DAN", "#LEG", "#CAM", "#HERA", "#LAC", "#PHI", "#FAB", "#MER", "#PAY", "#DEP"]
# Klasse "gruppe_wohlfahrtsausschuss" definieren
gruppe_wohlfahrtsausschuss = ["#ROB", "#STJU ", "#BAR", "#CAL", "#BIL"]
# Klasse "gruppe_sicherheitsausschuss" definieren
gruppe_sicherheitsausschuss = ["#AMA", "#VOU"]
# Klasse "gruppe_revolutionstribunal" definieren
gruppe_revolutionstribunal = ["#PRAES", "#DUM", "#FOU"]
# Klasse "gruppe_politik" definieren
gruppe_politik = gruppe_deputierte + gruppe_wohlfahrtsausschuss + gruppe_sicherheitsausschuss + gruppe_revolutionstribunal
# Klasse "gruppe_misc" definieren
gruppe_misc = ["#JUL", "#LUC", "#DIL", "#LAF", "#CHA", "#PAR", "#JACO", "#COL"]

# Einteilung durchführen
for person in liste_personen:
    temp = person
    # Klasse "politik"
    if person["who"] in gruppe_politik:
        temp["class"] = "politik"
    # Klasse "gruppe_misc"
    elif person["who"] in gruppe_misc:
        temp["class"] = "misc"
    # Klasse "False" wenn in keiner definierten Gruppe
    else:
        temp["class"] = False
    liste_personen_sorted.append(temp)

# Funktion: Ausgabe aller Personen einer bestimmten Klasse
def get_person_class(_class):
    print("==", _class)
    gesamt = [0,0]
    for person in liste_personen_sorted:
        if "class" in person and person["class"] == _class:
            gesamt[0] += person["count"]
            gesamt[1] += len(person['rede_full'].split())
            print(person["name"], person["count"], "|", len(person['rede_full'].split()))
    print("--> count_ges:", gesamt[0], "// rede_ges:", gesamt[1])

# Ausgabe der Personen der Klasse "politik"            
get_person_class("politik")
print("_____"*20)

# Ausgabe der Personen der Klasse "misc"            
get_person_class("misc")
print("_____"*20)

# Ausgabe der Personen ohne Klasse ("False")
get_person_class(False)

== politik
['Danton'] 117 | 4439
['Camille'] 46 | 1272
['Lacroix'] 43 | 1059
['Robespierre'] 29 | 2289
['Hérault'] 23 | 615
['St. Juſt'] 23 | 1006
['Barrère'] 17 | 466
['Präſident', 'Der Präſident', 'Hermann'] 16 | 243
['Philippeau'] 12 | 264
['Legendre'] 11 | 276
['Fouquier'] 11 | 188
['Mercier'] 9 | 186
['Payne'] 8 | 708
['Callot'] 6 | 131
['Billaud'] 5 | 79
['Dumas'] 5 | 93
['Eine Stimme', 'Stimme', 'Fabre'] 3 | 9
['Ein Deputirter'] 2 | 22
['Amar'] 2 | 21
['Voulaud'] 1 | 5
--> count_ges: 389 // rede_ges: 13371
____________________________________________________________________________________________________
== misc
['Julie'] 20 | 299
['Lucile'] 13 | 315
['Dillon'] 10 | 234
['Laflotte'] 10 | 310
['Chaumette'] 5 | 55
['Paris'] 3 | 53
['Ein Jacobiner', 'Die Jacobiner'] 2 | 13
['Collot d’Herbois'] 1 | 63
--> count_ges: 64 // rede_ges: 1342
____________________________________________________________________________________________________
== False
['Simon'] 23 | 309
['Weib', 'Ein Weib

In [38]:
## Wordcount für Person x
for person in liste_personen:
    if person['who'] == '#DAN' or person['who'] == '#ROB':
        print(person['name'])
        get_word_count(person["rede_full"])
        # print liste_personen

['Danton']
('revolution', 10)
('julie', 9)
('leben', 9)
('leute', 8)
('ruhe', 8)
('kind', 8)
('grab', 7)
('junge', 7)
('mühe', 6)
('auge', 6)
('erde', 6)
('stimme', 6)
('gedächtniß', 6)
('september', 6)
('tod', 6)
('hand', 5)
('leib', 5)
('robespierre', 5)
('guillotine', 5)
('himmel', 5)
('kopf', 5)
('mensch', 5)
('fenster', 5)
('haar', 4)
('sinn', 4)
('gedanke', 4)
('lippe', 4)
('freiheit', 4)
('nacht', 4)
('name', 4)
('volk', 4)
('werth', 4)
('rock', 4)
('republik', 4)
('gasse', 4)
('gliedern', 4)
('camille', 4)
('feind', 4)
('geschichte', 4)
('gott', 4)
('wort', 3)
('athem', 3)
('gesicht', 3)
('fuß', 3)
('freund', 3)
('mann', 3)
('mord', 3)
('natur', 3)
('bett', 3)
('ton', 3)
('adieu', 3)
('aristokrat', 3)
('stirne', 3)
('könig', 3)
('welt', 3)
('leiche', 3)
('schooß', 2)
('herz', 2)
('sarg', 2)
('stunde', 2)
-------------------------
['Robespierre']
('volk', 12)
('republik', 11)
('feind', 10)
('schrecken', 9)
('tugend', 8)
('erbarme', 7)
('gedanke', 7)
('revolution', 6)
('kraft', 5